<a href="https://colab.research.google.com/github/shikhar00778/toxic-comment-classification/blob/master/fastai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!kaggle competitions download -c jigsaw-toxic-comment-classification-challenge  

401 - Unauthorized


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp drive/'My Drive'/jigsaw-toxic-comment-classification-challenge.zip .

In [22]:
!unzip jigsaw-toxic-comment-classification-challenge.zip

Archive:  jigsaw-toxic-comment-classification-challenge.zip
  inflating: test_labels.csv         
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


download-all					   sample_submission.csv
drive						   test.csv
jigsaw-toxic-comment-classification-challenge.zip  test_labels.csv
sample_data					   train.csv


In [0]:
import pandas as pd
import numpy as np

In [0]:
df = pd.read_csv("train.csv")

In [0]:
arr = df.columns

In [32]:
classes = arr[2:]
classes = np.asarray(classes)
classes = classes.tolist()
classes

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [0]:
data=[]
lab=[]
for row,index in df.iterrows():
  temp=[]
  for class_name in classes:
    if index[class_name]==1:
      temp.append(class_name)
    if len(temp)==0: 
      temp.append("no_class")
  temp = ' '.join(temp)
  lab.append(temp)
  data.append(index["comment_text"])
X = pd.DataFrame({"text":data,"labels":lab})

In [40]:
X

,text,labels
0,Explanation\nWhy the edits made under my usern...,no_class
1,D'aww! He matches this background colour I'm s...,no_class
2,"Hey man, I'm really not trying to edit war. It...",no_class
3,"""\nMore\nI can't make any real suggestions on ...",no_class
4,"You, sir, are my hero. Any chance you remember...",no_class
5,"""\n\nCongratulations from me as well, use the ...",no_class
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,toxic severe_toxic obscene insult
7,Your vandalism to the Matt Shirvington article...,no_class
8,Sorry if the word 'nonsense' was offensive to ...,no_class
9,alignment on this subject and which are contra...,no_class


In [0]:
X.to_csv("train.csv",index=False)

In [0]:
from fastai.text import *
path = '.'

In [0]:
data_lm = TextDataBunch.from_csv(path, 'train.csv', text_col=0,label_cols=1,delim=" ")